In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/OPR NLP/

Mounted at /content/gdrive
/content/gdrive/My Drive/OPR NLP


In [2]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds

In [3]:
vehicles = pd.read_csv('20180901_20190101_allLocation.csv',index_col = False)
vehicles.head(2)

,EndLatitude,EndLongitude
0,38.262,-85.682
1,38.255,-85.781


In [4]:
v = np.array(vehicles[0:10])
n = 9

In [6]:
# read the documentation to transfer the value to longitude and latitude
!pip install pycrs
import pycrs
from pprint import pprint
crs = pycrs.load.from_file("./Dockless_Vehicle_Distribution_Zones/Dockless_Vehicle_Distribution_Zones.prj")

!pip install pyproj
from pyproj import Proj
myProj = Proj(crs.to_proj4())

# read coordinate files
!pip install PyShp
import shapefile
from shapely.geometry import Point, Polygon
from pprint import pprint
shape = shapefile.Reader("./Dockless_Vehicle_Distribution_Zones/Dockless_Vehicle_Distribution_Zones.shp")
#first feature of the shapefile
coordinate = {}
for i in range(9):
  feature = shape.shapeRecords()[i]
  coor= feature.shape.__geo_interface__ 
  coordinate[i] = coor['coordinates'][0]

# find the bound for the coordinate
coor_upper = []
coor_lower = []
x_range = []
y_range = []
for i in range(9):
  temp = pd.DataFrame(coordinate[i], columns = ['x','y'])
  x, y = myProj(temp.x,temp.y, inverse = True)
  x_max = x.max()
  x_min = x.min()
  y_max = y.max()
  y_min = y.min()
  coor_upper.extend([x_max, y_max])
  coor_lower.extend([x_min, y_min])
  x_range.append((x_min, x_max))
  y_range.append((y_min, y_max))



x_min = np.array(coor_lower).reshape(-1,2).T[0]
y_min = np.array(coor_lower).reshape(-1,2).T[1]
x_max = np.array(coor_upper).reshape(-1,2).T[0]
y_max = np.array(coor_upper).reshape(-1,2).T[1]

x_range_all = (x_min.min(),x_max.max())
y_range_all = (y_min.min(),y_max.max())

  Created wheel for pycrs: filename=PyCRS-1.0.2-cp36-none-any.whl size=32683 sha256=875ae34a597b9875ba94207929bdf00b22b0feb5c6f850e60f43d96fd1939ecd
  Stored in directory: /root/.cache/pip/wheels/1a/c9/56/648f762bd41fdb03454be51b8a7069795925168833cb61f316
Successfully built pycrs
     |████████████████████████████████| 6.5MB 8.8MB/s 
     |████████████████████████████████| 225kB 5.6MB/s 
  Created wheel for PyShp: filename=pyshp-2.1.2-cp36-none-any.whl size=36216 sha256=0271ba13e12af7e0eb83bcf48fb1e6fd953dd125c25a6b3c57d6dd86792de38b
  Stored in directory: /root/.cache/pip/wheels/96/6c/53/4112475adf3b831da97f083163d0f38ee6daac9c1b13f7afea
Successfully built PyShp


In [24]:
x_range_v = (vehicles.EndLongitude.min(),vehicles.EndLongitude.max())
y_range_v = (vehicles.EndLatitude.min(),vehicles.EndLatitude.max())
x_range_v, y_range_v


((-85.84200000000001, -85.569), (38.103, 38.371))

In [25]:
x_range_all, y_range_all

((-98.74424614038226, -98.53409122980943),
 (37.256575665827334, 37.41478947387985))

In [17]:
allzonerange = pd.DataFrame(np.array([list(x_min), list(x_max), list(y_min), list(y_max)]).T, columns = ['xmin','xmax','ymin','ymax'])
allzonerange.to_csv('all_zone_range.csv')

In [ ]:
!pip install pyomo
from pyomo.environ import *
from pyomo.dae import *
m= ConcreteModel()

# reference
# https://pyomo.readthedocs.io/en/stable/working_models.html
# https://projects.coin-or.org/Coopr/browser/pyomo/trunk/examples/dae/Heat_Conduction.py?rev=9315
# https://pyomo.readthedocs.io/en/stable/modeling_extensions/dae.html

In [ ]:
!pip install mip
from mip import Model, xsum, minimize, BINARY, CONTINUOUS

In [ ]:
from scipy.spatial import distance

In [ ]:
# x_range

[(-98.7440762869345, -98.67743592134778),
 (-98.74424614038226, -98.66923922836325),
 (-98.67885245865642, -98.6340392708933),
 (-98.6776637308838, -98.63113425473702),
 (-98.65620869197937, -98.59370364737312),
 (-98.65046095845308, -98.55635988801255),
 (-98.62115996442373, -98.53409122980943),
 (-98.73055268154587, -98.65839015751398),
 (-98.7232213007557, -98.64894479338597)]

In [ ]:
# model.x1 = Var(bounds=x_range[0])
# model.x2 = Var(bounds=x_range[1])

In [ ]:
v

array([[ 38.262, -85.682],
       [ 38.255, -85.781],
       [ 38.255, -85.763],
       [ 38.257, -85.758],
       [ 38.257, -85.757],
       [ 38.252, -85.696],
       [ 38.255, -85.744],
       [ 38.258, -85.716],
       [ 38.257, -85.757],
       [ 38.254, -85.782]])

In [ ]:
m = ConcreteModel()
'''variables'''
# binary
m.Nweight = Param(initialize=len(v)*9)
m.N = RangeSet(m.Nweight)
m.weight = Var(m.N, within = Binary)

# station coordination
m.SstationX = ContinuousSet(bounds = x_range_all)
m.SstationY = ContinuousSet(bounds = y_range_all)
m.stationX = Param(m.SstationX,initialize = 9)
m.stationY = Param(m.SstationY,initialize = 9)

# objective function
...


## Appendix
### Read shape data for first constraint

In [ ]:
!pip install PyShp
import shapefile
from shapely.geometry import Point, Polygon
shape = shapefile.Reader("./Dockless_Vehicle_Distribution_Zones/Dockless_Vehicle_Distribution_Zones.shp")
#first feature of the shapefile
coordinate = {}
for i in range(9):
  feature = shape.shapeRecords()[i]
  coor= feature.shape.__geo_interface__ 
  coordinate[i] = coor['coordinates'][0]

     |████████████████████████████████| 225kB 6.8MB/s 
  Created wheel for PyShp: filename=pyshp-2.1.2-cp36-none-any.whl size=36216 sha256=df8f73feb2b6ac6bfd00b181a0e5bb998fd6318483d49d539d636191b285d7b6
  Stored in directory: /root/.cache/pip/wheels/96/6c/53/4112475adf3b831da97f083163d0f38ee6daac9c1b13f7afea
Successfully built PyShp


True

In [ ]:
pd.DataFrame(coordinate[1], columns = ['x','y'])

,x,y
0,1.204659e+06,274362.969034
1,1.204581e+06,273864.339985
2,1.204477e+06,273211.377500
3,1.204445e+06,273006.591840
4,1.204422e+06,272861.552500
...,...,...
444,1.204848e+06,275203.351250
445,1.204775e+06,275046.838750
446,1.204750e+06,274902.667500
447,1.204701e+06,274628.125000


#### documentation to understand the prj file
http://www.geoapi.org/3.0/javadoc/org/opengis/referencing/doc-files/WKT.html

In [ ]:
# read the documentation to transfer the value to longitude and latitude
!pip install pycrs
import pycrs
from pprint import pprint
crs = pycrs.load.from_file("./Dockless_Vehicle_Distribution_Zones/Dockless_Vehicle_Distribution_Zones.prj")

<bound method ProjCS.to_ogc_wkt of <pycrs.elements.cs.ProjCS object at 0x7f39e8352780>>

#### crs.to_ogc_wkt()
    PROJCS["NAD_1983_StatePlane_Kentucky_North_FIPS_1601_Feet", 

        GEOGCS["GCS_North_American_1983", 

            DATUM["North_American_Datum_1983", 

                SPHEROID["GRS_1980", 6378137.0, 298.257222101]

            ], 

            PRIMEM["Greenwich", 0], 

            UNIT["degree", 0.0174532925199433], 

            AXIS["Lon", EAST], 

            AXIS["Lat", NORTH]

        ], 
        PROJECTION["Lambert_Conformal_Conic"], 

        PARAMETER["false_easting", 1640416.666666667], 

        PARAMETER["false_northing", 0.0], 

        PARAMETER["Central_Meridian", -84.25], 

        PARAMETER["standard_parallel_1", 37.96666666666667], 

        PARAMETER["standard_parallel_2", 38.96666666666667], 

        PARAMETER["latitude_of_origin", 37.5], 

        UNIT["Foot_US", 0.3048006096012192], 

        AXIS["X", EAST], AXIS["Y", NORTH]]

In [ ]:
crs.to_proj4()

'+proj=lcc +datum=NAD83 +ellps=GRS80 +a=6378137.0 +rf=298.257222101 +pm=0 +x_0=1640416.666666667 +y_0=0.0 +lon_0=-84.25 +lat_1=37.96666666666667 +lat_2=38.96666666666667 +lat_0=37.5 +units=us-ft +axis=enu +no_defs'

#### Transfer the proj to longitude and latitude

In [ ]:
!pip install pycrs
import pycrs
from pprint import pprint
crs = pycrs.load.from_file("./Dockless_Vehicle_Distribution_Zones/Dockless_Vehicle_Distribution_Zones.prj")

!pip install pyproj
from pyproj import Proj
myProj = Proj(crs.to_proj4())

In [ ]:
coordinate[7]

[(1198503.793750003, 266260.02249999344),
 (1198872.6875282228, 266218.88355474174),
 (1199091.4362500012, 266194.4887500107),
 (1199155.9058714807, 266180.91374632716),
 (1199463.3712500036, 266116.1724999994)]

In [ ]:
df = pd.DataFrame(coordinate[0], columns=['Foot_US_EAST','Foot_US_NORTH'])

lon, lat = myProj(df['Foot_US_EAST'].values, df['Foot_US_NORTH'].values, inverse = True)

In [ ]:
# test back
LCCx, LCCy = myProj(lon, lat)

In [ ]:

result = pd.DataFrame(np.c_[LCCx, LCCy, lon, lat], columns=['LCCx', 'LCCy', 'Lon', 'Lat'])
result.head()

,LCCx,LCCy,Lon,Lat
0,1.193692e+06,287815.076363,-98.722164,37.384932
1,1.193720e+06,287810.630178,-98.722065,37.384932
2,1.193756e+06,287807.880183,-98.721942,37.384940
3,1.193794e+06,287805.880187,-98.721811,37.384951
4,1.193831e+06,287804.130191,-98.721685,37.384962


In [ ]:
result.Lon.min()

-98.7440762869345

In [ ]:
def f(x):
    fval = (x[0] - x[1] - 1)**2
    
    return fval

def grad(x):
    fgrad = np.zeros(2)
    fgrad[0] = 2*(x[0] - x[1] - 1)
    fgrad[1] = -2*(x[0] - x[1] - 1)

    return fgrad

def con(x):
    cval = np.zeros(4)
    cval[0] = x[0]**2 - 2                       
    cval[1] = (x[0] - 1)**2 + (x[1] - 1)**2 - 3
    cval[2] = -(x[1] - x[0]**2 + 1)
    cval[3] = (x[1] - x[0]**2 + 1)*x[1]
    
    return -cval

In [ ]:
x0 = np.ones(2)
bounds = Bounds([-1,0],[2,np.inf])

cons = {'type':'ineq', 'fun': con}
res = minimize(f, x0, method='SLSQP', bounds=bounds, constraints=cons)
print(res.x)

[1.0000000e+00 4.4408921e-16]
